In [306]:
import pandas as pd
import numpy as np
import sympy as sp
import inspect
from datetime import datetime
from sympy import symbols, diff, exp, log, sqrt
from scipy.stats import norm
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Problem1

● Current Stock Price $151.03

● Strike Price $165

● Current Date 03/13/2022

● Options Expiration Date 04/15/2022

● Risk Free Rate of 4.25%

● Continuously Compounding Coupon of 0.53%

● Implied volatility 0.2

Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation.

Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. Assume
the stock above:

● Pays dividend on 4/11/2022 of $0.88

Calculate the value of the call and the put. Calculate the Greeks of each.
What is the sensitivity of the put and call to a change in the dividend amount?

In [142]:
S=151.03
X=165
current=datetime(2022, 3, 13)
expdate=datetime(2022, 4, 15)
ttm=(expdate-current).days/365

ivol=0.2
r=0.0425
q=0.0053
b=r-q

In [143]:
d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
d2 = d1 - ivol*np.sqrt(ttm)

In [144]:
def gbsm(ex, underlying, strike, ttm, rf, b, ivol):
    d1 = (np.log(underlying/strike) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)

    if ex=='Call':
        return underlying * np.exp((b-rf)*ttm) * norm.cdf(d1) - strike*np.exp(-rf*ttm)*norm.cdf(d2)
    else:
        return strike*np.exp(-rf*ttm)*norm.cdf(-d2) - underlying*np.exp((b-rf)*ttm)*norm.cdf(-d1)

In [145]:
def gbsm_delta(ex, S, X, ttm, ivol, r, b):
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    if ex=='Call':
        delta = norm.cdf(d1 , 0, 1) * np.exp((b-r)*ttm)
    else:
        delta = (norm.cdf(d1 , 0, 1) - 1) * np.exp((b-r)*ttm)
    
    return delta

def gbsm_gamma(ex, S, X, ttm, ivol, r, b):
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)
    gamma = norm.pdf(d1, 0, 1) / (S * ivol * np.sqrt(ttm))
    return gamma

def gbsm_vega(ex, S, X, ttm, ivol, r, b):
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)
    vega = S * norm.pdf(d1, 0, 1) * np.sqrt(ttm)
    return vega

def gbsm_theta(ex, S, X, ttm, ivol, r, b):
    is_call = 1 if ex == "Call" else -1
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)
    theta = -S * np.exp((b - r) * ttm) * norm.pdf(d1, 0, 1) * ivol / (2 * np.sqrt(ttm)) \
          -(b - r) * S * np.exp((b - r) * ttm) * norm.cdf(d1 * is_call, 0, 1) * is_call \
          -r * X * np.exp(-r * ttm) * norm.cdf(d2 * is_call, 0, 1) * is_call
    return theta

def gbsm_rho(ex, S, X, ttm, ivol, r, b):
    is_call = 1 if ex == "Call" else -1
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)
    rho = X * ttm * np.exp(-r * ttm) * norm.cdf(d2 * is_call, 0, 1) * is_call
    return rho

def gbsm_carry_rho(ex, S, X, ttm, ivol, r, b):
    is_call = 1 if ex == "Call" else -1
    d1 = (np.log(S/X) + (b+ivol**2/2)*ttm)/(ivol*np.sqrt(ttm))
    d2 = d1 - ivol*np.sqrt(ttm)
    carry_rho = S * ttm * np.exp((b - r) * ttm) * norm.cdf(d1 * is_call, 0, 1) * is_call
    return carry_rho

In [146]:
def first_order_der(func, x, delta):
  return (func(x + delta) - func(x - delta)) / (2 * delta)

def second_order_der(func, x, delta):
  return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
  arg_names = list(inspect.signature(func).parameters.keys())
  derivative_fs = {1: first_order_der, 2: second_order_der}

  def partial_derivative(*args, **kwargs):
    args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
    arg_val = args_dict.pop(arg_name)

    def partial_f(x):
      p_kwargs = {arg_name:x, **args_dict}
      return func(**p_kwargs)
    return derivative_fs[order](partial_f, arg_val, delta)
  return partial_derivative

In [147]:
# delta
delta_call = gbsm_delta("Call", S, X, ttm, ivol, r, b)
delta_put = gbsm_delta("Put", S, X, ttm, ivol, r, b)
gbsm_delta_num = cal_partial_derivative(gbsm, 1, 'underlying')
delta_call_num = gbsm_delta_num("Call", S, X, ttm, ivol, r, b)
delta_put_num = gbsm_delta_num("Put", S, X, ttm, ivol, r, b)
print(delta_call, delta_put)
print(delta_call_num, delta_put_num)

0.08297130333914773 -0.9165496333661425
1.986922887836227e-14 -0.9858611793589489


In [148]:
# gamma
gamma_call = gbsm_gamma("Call", S, X, ttm, ivol, r, b)
gamma_put = gbsm_gamma("Put", S, X, ttm, ivol, r, b)
gbsm_gamma_num = cal_partial_derivative(gbsm, 2, 'underlying')
gamma_call_num = gbsm_gamma_num("Call", S, X, ttm, ivol, r, b)
gamma_put_num = gbsm_gamma_num("Put", S, X, ttm, ivol, r, b)
print(gamma_call, gamma_put)
print(gamma_call_num, gamma_put_num)

0.016830979206204362 0.016830979206204362
9.042259959910318e-14 2.8421709430404007e-08


In [149]:
# vega
vega_call = gbsm_vega("Call", S, X, ttm, ivol, r, b)
vega_put = gbsm_vega("Put", S, X, ttm, ivol, r, b)
gbsm_vega_num = cal_partial_derivative(gbsm, 1, 'ivol')
vega_call_num = gbsm_vega_num("Call", S, X, ttm, ivol, r, b)
vega_put_num = gbsm_vega_num("Put", S, X, ttm, ivol, r, b)
print(vega_call, vega_put)
print(vega_call_num, vega_put_num)

6.942036604441163 6.942036604441163
9.767308509353247e-12 0.0


In [150]:
# theta
theta_call = gbsm_theta("Call", S, X, ttm, ivol, r, b)
theta_put = gbsm_theta("Put", S, X, ttm, ivol, r, b)
gbsm_theta_num = cal_partial_derivative(gbsm, 1, 'ttm')
theta_call_num = -gbsm_theta_num("Call", S, X, ttm, ivol, r, b)
theta_put_num = -gbsm_theta_num("Put", S, X, ttm, ivol, r, b)
print(theta_call, theta_put)
print(theta_call_num, theta_put_num)

-8.126522359668838 -1.9409914783019566
-1.5819627783921197e-12 8.957748660847642


In [151]:
# rho
rho_call = gbsm_rho("Call", S, X, ttm, ivol, r, b)
rho_put = gbsm_rho("Put", S, X, ttm, ivol, r, b)
gbsm_rho_num = cal_partial_derivative(gbsm, 1, 'rf')
rho_call_num = gbsm_rho_num("Call", S, X, ttm, ivol, r, b)
rho_put_num = gbsm_rho_num("Put", S, X, ttm, ivol, r, b)
print(rho_call, rho_put)
print(rho_call_num, rho_put_num)

1.1025939156368187 -13.758003122735788
-3.8791500416870894e-16 -1.1887809038029218


In [152]:
# carry rho
carry_rho_call = gbsm_carry_rho("Call", S, X, ttm, ivol, r, b)
carry_rho_put = gbsm_carry_rho("Put", S, X, ttm, ivol, r, b)
gbsm_carry_rho_num = cal_partial_derivative(gbsm, 1, 'b')
carry_rho_call_num = gbsm_carry_rho_num("Call", S, X, ttm, ivol, r, b)
carry_rho_put_num = gbsm_carry_rho_num("Put", S, X, ttm, ivol, r, b)
print(carry_rho_call, carry_rho_put)
print(carry_rho_call_num, carry_rho_put_num)

1.132953825011723 -12.515271800549371
2.7148101769271126e-13 -13.461704838221067


In [187]:
import math


def bt_american(call, underlying, strike, ttm, rf, b, ivol, N):
    dt = ttm / N
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(b * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = np.exp(-rf * dt)
    z = 1 if call else -1

    def n_node_func(n):
        return int((n + 1) * (n + 2) / 2)

    def idx_func(i, j):
        return n_node_func(j - 1) + i 

    n_nodes = n_node_func(N)
    option_values = np.empty(n_nodes, dtype=float)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idx_func(i, j)
            price = underlying * u ** i * d ** (j - i)
            option_values[idx] = max(0, z * (price - strike))

            if j < N:
                option_values[idx] = max(option_values[idx], df * (pu * option_values[idx_func(i + 1, j + 1)] + pd * option_values[idx_func(i, j + 1)]))

    return option_values[1]


In [208]:
def bt_american_div(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N):
    # If there are no dividends or the first dividend is outside our grid, return the standard bt_american value
    if not divAmts or not divTimes or divTimes[0] > N:
        return bt_american(call, underlying, strike, ttm, rf, rf, ivol, N)

    dt = ttm / N
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(rf * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = np.exp(-rf * dt)
    z = 1 if call else -1

    def n_node_func(n):
        return int((n + 1) * (n + 2) / 2)

    def idx_func(i, j):
        return n_node_func(j - 1) + i

    n_div = len(divTimes)
    n_nodes = n_node_func(divTimes[0])

    option_values = np.empty(n_nodes, dtype=float)

    for j in range(divTimes[0], -1, -1):
        for i in range(j, -1, -1):
            idx = idx_func(i, j)
            price = underlying * u ** i * d ** (j - i)

            if j < divTimes[0]:
                # Times before the dividend working backward induction
                option_values[idx] = max(0, z * (price - strike))
                option_values[idx] = max(option_values[idx], df * (pu * option_values[idx_func(i + 1, j + 1)] + pd * option_values[idx_func(i, j + 1)]))
            else:
                # Time of the dividend
                val_no_exercise = bt_american_div(call, price - divAmts[0], strike, ttm - divTimes[0] * dt, rf, divAmts[1:], [t - divTimes[0] for t in divTimes[1:]], ivol, N - divTimes[0])
                val_exercise = max(0, z * (price - strike))
                option_values[idx] = max(val_no_exercise, val_exercise)

    return option_values[0]

In [209]:
N=200
bt_no_div_call=bt_american(True, S, X, ttm, r, b, ivol, N)
bt_no_div_put=bt_american(False, S, X, ttm, r, b, ivol, N)
print('no div call:{}'.format(bt_no_div_call))
print('no div put:{}'.format(bt_no_div_put))

no div call:0.28273717030825374
no div put:14.649213091283734


In [210]:
div_date = datetime(2022, 4, 11)
div = [0.88]
div_time = [int((div_date - current).days / (expdate - current).days * N)]


value_call = bt_american_div(True, S, X, ttm, r, div, div_time, ivol, N)
value_put = bt_american_div(False, S, X, ttm, r, div, div_time, ivol, N)
print("Binomial tree value with dividend for call: " + str(value_call))
print("Binomial tree value with dividend for put: " + str(value_put))

Binomial tree value with dividend for call: 0.27886724160603915
Binomial tree value with dividend for put: 15.152886604860912


In [213]:
# delta
cal_amr_delta_num = cal_partial_derivative(bt_american_div, 1, 'underlying')
delta_call_amr = cal_amr_delta_num(True, S, X, ttm, r, div, div_time, ivol, N)
delta_put_amr = cal_amr_delta_num(False, S, X, ttm, r, div, div_time, ivol, N)
print(delta_call_amr, delta_put_amr)

0.07115919293726458 -0.9429524012087853


In [214]:
# gamma
cal_amr_gamma_num = cal_partial_derivative(bt_american_div, 2, 'underlying', delta=1)
gamma_call_amr = cal_amr_gamma_num(True, S, X, ttm, r, div, div_time, ivol, N)
gamma_put_amr = cal_amr_gamma_num(False, S, X, ttm, r, div, div_time, ivol, N)
print(gamma_call_amr, gamma_put_amr)

0.017208756516461143 0.015870488213149514


In [215]:
# vega
cal_amr_vega_num = cal_partial_derivative(bt_american_div, 1, 'ivol')
vega_call_amr = cal_amr_vega_num(True, S, X, ttm, r, div, div_time, ivol, N)
vega_put_amr = cal_amr_vega_num(False, S, X, ttm, r, div, div_time, ivol, N)
print(vega_call_amr, vega_put_amr)

6.065369195242698 8.031475027046042


In [216]:
# theta
cal_amr_theta_num = cal_partial_derivative(bt_american_div, 1, 'ttm')
theta_call_amr = -cal_amr_theta_num(True, S, X, ttm, r, div, div_time, ivol, N)
theta_put_amr = -cal_amr_theta_num(False, S, X, ttm, r, div, div_time, ivol, N)
print(theta_call_amr, theta_put_amr)

-7.149208870630114 -3.035580176777408


In [218]:
# rho
cal_amr_rho_num = cal_partial_derivative(bt_american_div, 1, 'rf')
rho_call_amr = cal_amr_rho_num(True, S, X, ttm, r, div, div_time, ivol, N)
rho_put_amr = cal_amr_rho_num(False, S, X, ttm, r, div, div_time, ivol, N)
print(rho_call_amr, rho_put_amr)

0.8778747502689588 -12.502269537795208


In [224]:
delta = 1e-3

div_p, div_m=[],[]
for m in div:
    div_p.append(m+delta)
    div_m.append(m-delta)
call_value1 = bt_american_div(True, S, X, ttm, r, div_p, div_time, ivol, N)    
call_value2 = bt_american_div(True, S, X, ttm, r, div_m, div_time, ivol, N)    
call_sens_to_div_amount = (call_value1 - call_value2) / (2*delta)


put_value1 = bt_american_div(False, S, X, ttm, r, div_p, div_time, ivol, N) 
put_value2 = bt_american_div(False, S, X, ttm, r, div_m, div_time, ivol, N)    
put_sens_to_div_amount = (put_value1 - put_value2) / (2*delta)
print(f"Sensitivity to dividend amount: Call: {call_sens_to_div_amount:.3f}, Put: {put_sens_to_div_amount:.3f}")

Sensitivity to dividend amount: Call: -0.013, Put: 0.945


# Problem2

Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and
assuming :

● American Options

● Current Date 03/03/2023

● Current AAPL price is 165

● Risk Free Rate of 4.25%

● Dividend Payment of $1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL 
returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR 
and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a $ loss, not percentages.

Compare these results to last week’s results.

In [225]:
def cal_ES(x,alpha=0.05):
    xs = np.sort(x)
    n = alpha * len(xs)
    iup = math.ceil(n)
    idn = math.floor(n)
    VaR = (xs[iup] + xs[idn]) / 2
    ES = xs[0:idn].mean()
    return VaR,ES

In [246]:
def implied_vol_american(call, S0, X, ttm, div_time, div, r, N, market_price, x0=0.5):
  def equation(sigma):
    return bt_american_div(call, S0, X, ttm, r, div, div_time, ivol, N) - market_price
  # Back solve the binomial tree valuation to get the implied volatility
  return scipy.optimize.fsolve(equation, x0=x0, xtol=0.00001)[0]

def calculate_sim_values(portfolios, sim_prices, days_ahead=0):
  sim_values = pd.DataFrame(index=portfolios.index, 
                            columns=list(range(sim_prices.shape[0])))
  sim_prices = np.array(sim_prices)
  for i in portfolios.index:
    if portfolios["Type"][i] == "Stock":
      # For stock, the single value is its price
      single_values = sim_prices
    else:
      # For option, calculate values with gbsm method
      call = (portfolios["OptionType"][i]=='Call')
      X = portfolios["Strike"][i]
      ttm = ((portfolios["ExpirationDate"][i] - current_date).days - days_ahead) / 365
      ivol = portfolios["ImpliedVol"][i]
      div_time = [int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)]
      div = [1]
      option_values = []
      for S in sim_prices:
        option_values.append(bt_american_div(call, S, X, ttm, r, div, div_time, ivol, N))
      single_values = np.array(option_values)
    
    # Calculate the total values based on holding
    sim_values.loc[i, :] = portfolios["Holding"][i] * single_values
  
  # Combine the values for same portfolios
  sim_values['Portfolio'] = portfolios['Portfolio']
  return sim_values.groupby('Portfolio').sum()

In [247]:
S = 151.03
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = [1]

portfolios = pd.read_csv('problem2.csv', parse_dates=['ExpirationDate'])
portfolios['CurrentValue'] = portfolios['CurrentPrice'] * portfolios['Holding']


implied_vols = []
for i in range(len(portfolios.index)):
  if portfolios["Type"][i] == "Stock":
    implied_vols.append(None)
  else:
    option_type = (portfolios["OptionType"][i]=='Call')
    X = portfolios["Strike"][i]
    T = (portfolios["ExpirationDate"][i] - current_date).days / 365
    div_time = [int((div_date - current_date).days / (portfolios["ExpirationDate"][i] - current_date).days * N)]
    market_price = portfolios["CurrentPrice"][i]
    sigma = implied_vol_american(option_type, S, X, T, div_time, div, r, N, market_price)
    implied_vols.append(sigma)

# Store the implied volatility in portfolios
portfolios["ImpliedVol"] = implied_vols

/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [255]:
sim_prices = np.linspace(50, 200, 50)

sim_values = calculate_sim_values(portfolios, sim_prices, 10)

In [266]:
S = 151.03
N = 25
current_date = datetime(2023, 3, 3)
div_date = datetime(2023, 3, 15)
r = 0.0425
div = 1

prices = pd.read_csv("DailyPrices.csv")
prices = prices.drop(['Date'], axis=1)
returns=prices.pct_change().dropna()

# Simulate the prices based on returns with normal distribution
std = returns['AAPL'].std()
np.random.seed(42)
sim_returns = scipy.stats.norm(0, std).rvs((10, 100))
sim_prices = S * (1 + sim_returns).prod(axis=0)

In [268]:
# Calculate the current values and sim values
portfolios["CurrentValue"] = portfolios["CurrentPrice"] * portfolios["Holding"]
curr_values = portfolios.groupby('Portfolio')['CurrentValue'].sum()
sim_values = calculate_sim_values(portfolios, sim_prices, 10)

In [270]:
curr_values

Portfolio
Call              6.80
CallSpread        4.59
CoveredCall     146.98
ProtectedPut    154.04
Put               4.85
PutSpread         3.01
Stock           151.03
Straddle         11.65
SynLong           1.95
Name: CurrentValue, dtype: float64

In [269]:
sim_values

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Portfolio,,,,,,,,,,,,,,,,,,,,,
Call,6.953101,7.750782,9.736275,16.205418,2.705839,11.142000,8.338365,8.916344,6.247833,31.042650,...,7.418478,13.299328,5.142202,8.404274,16.272205,6.090158,6.507007,9.919765,9.758567,6.360355
CallSpread,3.194590,3.484015,4.254002,6.235450,1.440713,4.747332,3.696835,3.926298,2.938695,8.946456,...,3.363444,5.390170,2.504290,3.721638,6.254517,2.881486,3.032732,4.327338,4.262912,2.979522
CoveredCall,143.859532,144.967294,146.960972,151.037572,135.036192,148.193925,145.667297,146.203569,142.880105,154.264432,...,144.505814,149.601864,140.990326,145.730549,151.064802,142.661139,143.240028,147.130469,146.981564,143.036368
ProtectedPut,158.833411,159.639653,161.937569,168.373266,153.790304,163.423076,160.383147,161.026609,158.121361,182.186727,...,159.303761,165.449987,156.817098,160.458909,168.437856,157.962302,158.382814,162.141654,161.962350,158.234871
Put,12.865039,11.984442,10.054498,6.288709,20.554520,8.928652,11.337104,10.805072,13.647015,2.391270,...,12.350620,7.751560,15.304870,11.274352,6.258074,13.822802,13.358842,9.886643,10.034104,13.521769
PutSpread,5.508765,5.244565,4.667806,3.245065,7.437494,4.239030,5.051068,4.892044,5.744174,1.429381,...,5.354065,3.794381,6.244723,5.032318,3.233572,5.797295,5.656876,4.617732,4.661722,5.706148
Stock,148.972681,150.656023,154.354833,163.891169,137.005519,156.698261,151.895995,152.915266,147.484357,180.625923,...,149.954763,159.894280,144.755036,152.016215,163.974953,147.151618,148.031291,154.676990,154.393972,147.721812
Straddle,19.818140,19.735224,19.790772,22.494126,23.260358,20.070652,19.675468,19.721417,19.894847,33.433920,...,19.769098,21.050888,20.447072,19.678626,22.530279,19.912960,19.865849,19.806408,19.792671,19.882123
SynLong,-5.911938,-4.233659,-0.318223,9.916709,-17.848681,2.213348,-2.998739,-1.888728,-7.399182,28.651380,...,-4.932142,5.547768,-10.162667,-2.870077,10.014131,-7.732644,-6.851836,0.033121,-0.275538,-7.161414


In [280]:
result=pd.DataFrame(0, index=sim_value_changes.index, columns=['Mean', 'VaR', 'ES'])
for i in range(len(sim_value_changes.index.tolist())):
    result.iloc[i, 0]=index=sim_value_changes.iloc[i, :].mean()
    result.iloc[i, 1], result.iloc[i, 2]=cal_ES(sim_value_changes.iloc[i, :])
result

,Mean,VaR,ES
Portfolio,,,
Call,2.364587,-4.220267,-5.159150
CallSpread,-0.736854,-3.201527,-3.670549
CoveredCall,-2.370038,-12.486353,-17.105050
ProtectedPut,6.955003,-0.464203,-1.933145
Put,7.284255,0.735192,-0.879949
PutSpread,2.112575,-0.029255,-0.813042
Stock,0.599027,-14.682045,-20.050934
Straddle,9.648843,8.042447,8.031390
SynLong,-4.919668,-20.443891,-25.697596


In [296]:
holdings=portfolios.groupby('Portfolio')['Holding'].sum()

In [297]:
result=pd.DataFrame(0, index=sim_value_changes.index, columns=['Mean', 'VaR', 'ES'])
for i in range(len(sim_value_changes.index.tolist())):
    result.iloc[i, 0]=0
    result.iloc[i, 1]=del_norm_VaR(curr_values, holdings, sim_returns, lamda=0.94, alpha=0.05)
result

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

# Problem3

Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the
Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following
stocks.

AAPL FB UNH MA
MSFT NVDA HD PFE
AMZN BRK-B PG XOM
TSLA JPM V DIS
GOOGL JNJ BAC CSCO

Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100)
to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock.
Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio.

In [321]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
data = ff.join(mom, how='right') / 100

all_prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date']).set_index('Date')
all_returns=all_prices.pct_change().dropna()
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'RF']
dataset = all_returns[stocks].join(data)

data

,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,
1926-11-03,0.0020,-0.0020,-0.0033,0.00013,0.0056
1926-11-04,0.0059,-0.0012,0.0065,0.00013,-0.0050
1926-11-05,0.0007,-0.0011,0.0026,0.00013,0.0117
1926-11-06,0.0016,-0.0029,0.0005,0.00013,-0.0003
1926-11-08,0.0052,-0.0012,0.0018,0.00013,-0.0001
...,...,...,...,...,...
2023-01-25,0.0000,-0.0004,0.0065,0.00017,0.0014
2023-01-26,0.0108,-0.0058,0.0001,0.00017,-0.0123
2023-01-27,0.0036,0.0062,-0.0116,0.00017,-0.0246


In [320]:
avg_factor_rets = data.loc['2013-1-31':'2023-1-31'].mean(axis=0)
avg_daily_rets = pd.Series()
for stock in stocks:
  model = sm.OLS(dataset[stock] - dataset['RF'], sm.add_constant(dataset[factors]))
  results = model.fit()
    
  avg_daily_rets[stock] = (results.params[factors] * avg_factor_rets[factors]).sum() \
                          + avg_factor_rets['RF']

geo_means = np.log(1 + avg_daily_rets) * 255  
geo_covariance = np.log(1 + all_returns[stocks]).cov() * 255
arith_means = np.exp(geo_means + np.diagonal(geo_covariance.values) / 2) - 1

nstocks = geo_covariance.shape[0]
arith_covariance = np.empty((nstocks, nstocks), dtype=float)
for i in range(nstocks):
  for j in range(i, nstocks):
    mu_i, mu_j = geo_means.iloc[i], geo_means.iloc[j]
    sigma2_i, sigma2_j = geo_covariance.iloc[i, i], geo_covariance.iloc[j, j]
    sigma_ij = geo_covariance.iloc[i, j]
    arith_covariance[i, j] = np.exp(mu_i + mu_j + (sigma2_i + sigma2_j) / 2) * (np.exp(sigma_ij) - 1)
    arith_covariance[j, i] = arith_covariance[i, j]
arith_covariance = pd.DataFrame(arith_covariance, columns=stocks, index=stocks)

<ipython-input-320-6fc9e13f4fb7>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  avg_daily_rets = pd.Series()


MissingDataError: exog contains inf or nans